<a href="https://colab.research.google.com/github/alexlautw9527/for_notebook/blob/main/Fund2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np

In [2]:
fund_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/基金產品列表.csv', encoding='CP950')

In [3]:
fund_code_df = pd.read_csv('/content/drive/MyDrive/05_OTHER_DATA/FUND_CODE_DATA.csv', encoding='CP950')
fund_code_df = fund_code_df[['產品代碼', '基金名稱', 'ISIN Code']]

In [4]:
fund_df = fund_df[fund_df['RISKLEVEL']!=0][['CNAME','FUNDCRYID', 'RISKLEVEL','DIVIDENDTYPE2','ASSETCAT_CODE','SECTOR_NAME']]
fund_df['SECTOR_NAME'] = fund_df['SECTOR_NAME'].fillna('NA')
fund_df = fund_df[~fund_df['ASSETCAT_CODE'].isna()]

In [5]:
corpus = fund_df.astype(str).values.tolist()

In [9]:
corpus[0:5]

[['施羅德２０２２到期新興市場主權債券基金（美元）－累積型（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '03_不配',
  'BOND',
  'NA'],
 ['施羅德２０２２到期新興市場主權債券基金（美元）－配息型（本基金進行配息前未先扣除應負擔之相關費用）（本基金有相當比重投資於非投資等級之高風險債券且配息來源可能為本金）',
  'USD',
  '3',
  '01_配息',
  'BOND',
  'NA'],
 ['新光創新科技基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '科技'],
 ['新光店頭基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '台灣股市'],
 ['新光台灣富貴基金（新臺幣）', 'TWD', '4', '03_不配', 'STOCK', '台灣股市']]

In [10]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [11]:
model = Word2Vec(corpus, size=50, iter=50,window=5, sg=1,negative=20,min_count=1)


In [13]:
def most_similar(w2v_model, words, topn=20):
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

In [24]:
np.savetxt("fund_embedd.tsv", model.wv[fund_df['CNAME'].tolist()], delimiter="\t")


In [25]:
fund_df.to_csv('fund_embedd_metadata.tsv', sep = '\t',index=False)

In [26]:
fund_df.shape

(2650, 6)

In [27]:
from google.colab import files
files.download('/content/fund_embedd_metadata.tsv') 
files.download('/content/fund_embedd.tsv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>